In [1]:
import sqlite3
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
conn = sqlite3.connect('Northwind_small.sqlite')

## Question 1
### Does discount amount have a statistically significant effect on the quantity of a product in an order? If so, at what level(s) of discount?

In [3]:
x = """SELECT Quantity, Discount
       FROM OrderDetail
       ;"""
q1A = pd.read_sql_query(x, conn)
q1A.head()

,Quantity,Discount
0,12,0.0
1,10,0.0
2,5,0.0
3,9,0.0
4,40,0.0


In [4]:
for ind, x in enumerate(q1A.Discount):
    if x == 0:
        q1A.loc[ind, 'is_Discount'] = False
    else:
        q1A.loc[ind, 'is_Discount'] = True
q1A.sample(20)

,Quantity,Discount,is_Discount
58,20,0.05,True
851,35,0.15,True
265,10,0.00,False
638,20,0.20,True
671,20,0.00,False
150,30,0.00,False
1609,42,0.00,False
1567,30,0.00,False
760,10,0.20,True
1161,30,0.00,False


In [5]:
f = 'Quantity ~ C(is_Discount)'
lm = ols(f, q1A).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                       sum_sq      df          F        PR(>F)
C(is_Discount)   14903.513199     1.0  41.971781  1.144092e-10
Residual        764496.122996  2153.0        NaN           NaN


In [6]:
q1A.Discount.value_counts()

0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Discount, dtype: int64

In [7]:
q1B = q1A.loc[q1A['Discount'] != 0]
q1B = q1B.reset_index(drop=True)

In [8]:
newc = ['Discountlow', 'Discount10', 'Discount15', 'Discount20', 'Discount25']
for x in newc:
    q1B[x] = False

for idx, x in enumerate(q1B.Discount):
    if 0 < x <= 0.06:
        q1B.loc[idx, 'Discountlow'] = True
    elif x == 0.10:
        q1B.loc[idx, 'Discount10'] = True
    elif x == 0.15:
        q1B.loc[idx, 'Discount15'] = True
    elif x == 0.20:
        q1B.loc[idx, 'Discount20'] = True
    elif x == 0.25:
        q1B.loc[idx, 'Discount25'] = True
q1B.sample()
display(q1B.Quantity.value_counts())
q1B.Quantity.describe()

20     98
30     72
15     69
40     53
10     51
25     36
50     35
12     35
35     29
60     28
6      25
18     23
24     22
2      22
8      21
21     20
70     18
4      16
5      14
16     14
36     12
3      12
42     11
28     11
80     10
14      8
9       8
7       6
100     6
55      5
65      5
45      5
120     5
1       4
49      4
90      3
11      2
32      2
13      2
27      2
44      2
48      2
56      2
77      2
84      2
33      1
54      1
66      1
130     1
Name: Quantity, dtype: int64

count    838.000000
mean      27.109785
std       20.771439
min        1.000000
25%       12.000000
50%       20.000000
75%       36.000000
max      130.000000
Name: Quantity, dtype: float64

In [9]:
z = q1B['Quantity'].value_counts()
z1 = z.to_dict()
q1B['Count'] = q1B['Quantity'].map(z1)
for idx, x in enumerate(q1B.Count):
    if x < 4:
        if q1B.Quantity[idx] == 1:
            pass
        else:
            q1B.drop(index = idx, axis = 0, inplace=True)
q1B = q1B.reset_index(drop=True)

In [10]:
q1B.head()

,Quantity,Discount,is_Discount,Discountlow,Discount10,Discount15,Discount20,Discount25,Count
0,35,0.15,True,False,False,True,False,False,29
1,15,0.15,True,False,False,True,False,False,69
2,6,0.05,True,True,False,False,False,False,25
3,15,0.05,True,True,False,False,False,False,69
4,40,0.05,True,True,False,False,False,False,53


In [11]:
f = 'Quantity ~ C(Discountlow) + C(Discount10) + C(Discount15) + C(Discount20) + C(Discount25)'
lm = ols(f, q1B).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                       sum_sq     df          F    PR(>F)
C(Discountlow)    3859.920738    1.0   9.776806  0.001831
C(Discount10)     1048.339618    1.0   2.655343  0.103592
C(Discount15)     4808.926912    1.0  12.180547  0.000509
C(Discount20)     3710.582189    1.0   9.398546  0.002244
C(Discount25)     7210.642740    1.0  18.263861  0.000022
Residual        319001.510225  808.0        NaN       NaN


## Question 2
### Is there a significant difference in the level(s) of discount offered by UK employees and USA employees?

In [12]:
x = """SELECT od.Quantity, od.Discount, e.Country
       FROM OrderDetail AS od
       JOIN [Order] ON od.OrderId = [Order].Id
       JOIN Employee AS e ON [Order].EmployeeId = e.Id
       ;"""
q3 = pd.read_sql_query(x, conn)
q3.head()

,Quantity,Discount,Country
0,12,0.0,UK
1,10,0.0,UK
2,5,0.0,UK
3,9,0.0,UK
4,40,0.0,UK


In [13]:
f = 'Discount ~ C(Country)'
lm = ols(f, q3).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq      df         F    PR(>F)
C(Country)   0.067081     1.0  9.671415  0.001896
Residual    14.933259  2153.0       NaN       NaN


In [14]:
q3A = q3.drop('Quantity', axis = 1).pivot_table(index='Country', columns='Discount', aggfunc=len, fill_value=0, margins=True)
q3A

Discount,0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.1,0.15,0.2,0.25,All
Country,,,,,,,,,,,,
UK,309,0,0,0,0,55,0,60,53,45,46,568
USA,1008,1,2,3,1,130,1,113,104,116,108,1587
All,1317,1,2,3,1,185,1,173,157,161,154,2155


In [15]:
for x in q3A.columns:
    q3A.loc['UK', x] = round((float(q3A[x]['UK'])/float(q3A['All']['UK'])*100), 2)
    q3A.loc['USA', x] = round((float(q3A[x]['USA'])/float(q3A['All']['USA'])*100), 2)
q3A.drop(['All'], axis = 1, inplace = True)
q3A.drop(['All'], axis = 0, inplace = True)
q3A

Discount,0.0,0.01,0.02,0.03,0.04,0.05,0.06,0.1,0.15,0.2,0.25
Country,,,,,,,,,,,
UK,54.40,0.00,0.00,0.00,0.00,9.68,0.00,10.56,9.33,7.92,8.10
USA,63.52,0.06,0.13,0.19,0.06,8.19,0.06,7.12,6.55,7.31,6.81


## Question 3
### Does discount amount have a statistically significant effect on the quantity of a product in an order in the US? If so, at what level(s) of discount?

In [16]:
for x in newc:
    q3[x] = False

for idx, x in enumerate(q3.Discount):
    if 0 < x <= 0.06:
        q3.loc[idx, 'Discountlow'] = True
    elif x == 0.10:
        q3.loc[idx, 'Discount10'] = True
    elif x == 0.15:
        q3.loc[idx, 'Discount15'] = True
    elif x == 0.20:
        q3.loc[idx, 'Discount20'] = True
    elif x == 0.25:
        q3.loc[idx, 'Discount25'] = True
q3.sample(20)

,Quantity,Discount,Country,Discountlow,Discount10,Discount15,Discount20,Discount25
821,20,0.05,USA,True,False,False,False,False
1427,14,0.00,USA,False,False,False,False,False
1062,2,0.25,USA,False,False,False,False,True
276,77,0.05,USA,True,False,False,False,False
635,30,0.00,USA,False,False,False,False,False
69,33,0.05,USA,True,False,False,False,False
1615,8,0.00,USA,False,False,False,False,False
1832,24,0.15,UK,False,False,True,False,False
1367,40,0.00,USA,False,False,False,False,False
1574,55,0.20,USA,False,False,False,True,False


In [17]:
q3B = q3.loc[q3['Discount'] != 0]
q3B = q3B.reset_index(drop=True)
q3B.head()

,Quantity,Discount,Country,Discountlow,Discount10,Discount15,Discount20,Discount25
0,35,0.15,USA,False,False,True,False,False
1,15,0.15,USA,False,False,True,False,False
2,6,0.05,USA,True,False,False,False,False
3,15,0.05,USA,True,False,False,False,False
4,40,0.05,USA,True,False,False,False,False


In [18]:
z = q3B['Quantity'].value_counts()
z1 = z.to_dict()
q3B['Count'] = q3B['Quantity'].map(z1)
for idx, x in enumerate(q3B.Count):
    if x < 4:
        if q3B.Quantity[idx] == 1:
            pass
        else:
            q3B.drop(index = idx, axis = 0, inplace=True)
q3B = q3B.reset_index(drop=True)
q3B.head()

,Quantity,Discount,Country,Discountlow,Discount10,Discount15,Discount20,Discount25,Count
0,35,0.15,USA,False,False,True,False,False,29
1,15,0.15,USA,False,False,True,False,False,69
2,6,0.05,USA,True,False,False,False,False,25
3,15,0.05,USA,True,False,False,False,False,69
4,40,0.05,USA,True,False,False,False,False,53


In [19]:
q3USA = q3B.loc[q3B['Country'] != 'UK']
q3USA = q3USA.reset_index(drop=True)
q3USA.head()

,Quantity,Discount,Country,Discountlow,Discount10,Discount15,Discount20,Discount25,Count
0,35,0.15,USA,False,False,True,False,False,29
1,15,0.15,USA,False,False,True,False,False,69
2,6,0.05,USA,True,False,False,False,False,25
3,15,0.05,USA,True,False,False,False,False,69
4,40,0.05,USA,True,False,False,False,False,53


In [20]:
f = 'Quantity ~ C(Discountlow) + C(Discount10) + C(Discount15) + C(Discount20) + C(Discount25)'
lm = ols(f, q3USA).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                       sum_sq     df         F    PR(>F)
C(Discountlow)    3068.541493    1.0  7.526524  0.006276
C(Discount10)     1698.908746    1.0  4.167086  0.041687
C(Discount15)     3675.837972    1.0  9.016101  0.002796
C(Discount20)     2832.600694    1.0  6.947808  0.008626
C(Discount25)     2184.416458    1.0  5.357940  0.020991
Residual        227087.259134  557.0       NaN       NaN


### Does discount amount have a statistically significant effect on the quantity of a product in an order in the UK? If so, at what level(s) of discount?

In [21]:
q3UK = q3B.loc[q3B['Country'] != 'USA']
q3UK = q3UK.reset_index(drop=True)
q3UK.head()

,Quantity,Discount,Country,Discountlow,Discount10,Discount15,Discount20,Discount25,Count
0,15,0.15,UK,False,False,True,False,False,69
1,21,0.15,UK,False,False,True,False,False,20
2,60,0.25,UK,False,False,False,False,True,28
3,60,0.25,UK,False,False,False,False,True,28
4,36,0.25,UK,False,False,False,False,True,12


In [22]:
f = 'Quantity ~ C(Discountlow) + C(Discount10) + C(Discount15) + C(Discount20) + C(Discount25)'
lm = ols(f, q3UK).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                      sum_sq     df          F    PR(>F)
C(Discountlow)    759.507299    1.0   2.083938  0.150128
C(Discount10)       7.170052    1.0   0.019673  0.888568
C(Discount15)    1096.591752    1.0   3.008831  0.084065
C(Discount20)     831.541590    1.0   2.281586  0.132201
C(Discount25)    6964.440564    1.0  19.109051  0.000018
Residual        89656.592439  246.0        NaN       NaN


## Question 3

In [67]:
x = """SELECT [Order].OrderDate, ca.CategoryName, od.Quantity
       FROM OrderDetail AS od
       JOIN [Order] ON od.OrderId = [Order].Id
       JOIN Product AS p ON od.ProductId = p.Id
       JOIN Customer AS c ON c.Id = [Order].CustomerId
       JOIN Category AS ca ON p.CategoryId = ca.Id
       ;"""
q4 = pd.read_sql_query(x, conn)
q4.head()

,OrderDate,CategoryName,Quantity
0,2012-07-04,Dairy Products,12
1,2012-07-04,Grains/Cereals,10
2,2012-07-04,Dairy Products,5
3,2012-07-05,Produce,9
4,2012-07-05,Produce,40


In [68]:
q4.OrderDate = pd.to_datetime(q4.OrderDate)
#create dictionary for seasons
season_names = {1: 'Winter', 2 : 'Spring', 3: 'Summer', 4: 'Fall'}
#using modular arithmetic, we split the year into 4 3-month sections and convert each
#month for each row to a number between 1-4
season = q4.OrderDate.apply(lambda dt: (dt.month%12 + 3)//3)
#map to our dictionary
q4['season'] = season.map(season_names)
for idx, x in enumerate(q4.season):
    q4.loc[idx, 'Winter'] = True if x == 'Winter' else False
    q4.loc[idx, 'Spring'] = True if x == 'Spring' else False
    q4.loc[idx, 'Summer'] = True if x == 'Summer' else False
    q4.loc[idx, 'Fall'] = True if x == 'Fall' else False
q4.sample(10)

,OrderDate,CategoryName,Quantity,season,Winter,Spring,Summer,Fall
331,2012-12-06,Produce,15,Winter,True,False,False,False
267,2012-11-07,Beverages,15,Fall,False,False,False,True
348,2012-12-13,Dairy Products,60,Winter,True,False,False,False
2053,2014-05-06,Beverages,24,Spring,False,True,False,False
112,2012-08-27,Meat/Poultry,15,Summer,False,False,True,False
880,2013-07-07,Beverages,20,Summer,False,False,True,False
1019,2013-08-27,Beverages,20,Summer,False,False,True,False
1147,2013-10-10,Beverages,5,Fall,False,False,False,True
171,2012-09-23,Beverages,24,Fall,False,False,False,True
1353,2013-12-18,Dairy Products,10,Winter,True,False,False,False


In [71]:
z = q4['Quantity'].value_counts()
z1 = z.to_dict()
q4['Count'] = q4['Quantity'].map(z1)
for idx, x in enumerate(q4.Count):
    if x < 4:
        if q4.Quantity[idx] == 1:
            pass
        else:
            q4.drop(index = idx, axis = 0, inplace=True)
q4 = q4.reset_index(drop=True)
q4.sample(10)

,OrderDate,CategoryName,Quantity,season,Winter,Spring,Summer,Fall,Count
291,2012-11-22,Produce,30,Fall,False,False,False,True,185
1103,2013-10-01,Beverages,14,Fall,False,False,False,True,35
1930,2014-04-20,Confections,15,Spring,False,True,False,False,159
1611,2014-02-18,Seafood,28,Winter,True,False,False,False,28
1367,2013-12-25,Dairy Products,24,Winter,True,False,False,False,54
1628,2014-02-24,Confections,20,Winter,True,False,False,False,247
1077,2013-09-22,Confections,7,Fall,False,False,False,True,22
1815,2014-03-31,Produce,60,Spring,False,True,False,False,56
1335,2013-12-17,Dairy Products,1,Winter,True,False,False,False,17
1529,2014-01-29,Produce,5,Winter,True,False,False,False,62


In [72]:
newc = ['Dairy Products', 'Grains/Cereals', 'Produce', 'Seafood',
       'Condiments', 'Confections', 'Beverages', 'Meat/Poultry']
for x in newc:
    print(x)
    q41 = q4.loc[q4['CategoryName'] == x]
    q41 = q41.reset_index(drop=True)
    for y in q41.columns[4:8]:
        print(y + ':')
        f = 'Quantity ~ C(' + y + ')'
        lm = ols(f, q41).fit()
        table = sm.stats.anova_lm(lm, typ=2)
        print(table)
        print()
        print()

print('Total')
for y in q4.columns[4:8]:
    print(y + ':')
    f = 'Quantity ~ C(' + y + ')'
    lm = ols(f, q4).fit()
    table = sm.stats.anova_lm(lm, typ=2)
    print(table)
    print()
    print()

Dairy Products
Winter:
                  sum_sq     df         F    PR(>F)
C(Winter)    1704.421018    1.0  5.330455  0.021546
Residual   110314.270625  345.0       NaN       NaN


Spring:
                  sum_sq     df         F   PR(>F)
C(Spring)    1102.695478    1.0  3.429892  0.06488
Residual   110915.996165  345.0       NaN      NaN


Summer:
                  sum_sq     df         F    PR(>F)
C(Summer)       3.894425    1.0  0.011995  0.912854
Residual   112014.797218  345.0       NaN       NaN


Fall:
                 sum_sq     df        F    PR(>F)
C(Fall)       27.358309    1.0  0.08428  0.771754
Residual  111991.333333  345.0      NaN       NaN


Grains/Cereals
Winter:
                 sum_sq     df         F    PR(>F)
C(Winter)    230.122440    1.0  0.856388  0.355951
Residual   49980.579687  186.0       NaN       NaN


Spring:
                 sum_sq     df         F    PR(>F)
C(Spring)    358.550990    1.0  1.337765  0.248912
Residual   49852.151138  186.0       NaN    

## Question 4

In [104]:
x = """SELECT od.OrderId, [Order].ShipVia, [Order].Freight
       FROM OrderDetail AS od
       JOIN [Order] ON od.OrderId = [Order].Id
       ;"""
q5 = pd.read_sql_query(x, conn)
q5

,OrderId,ShipVia,Freight
0,10248,3,32.38
1,10248,3,32.38
2,10248,3,32.38
3,10249,1,11.61
4,10249,1,11.61
5,10250,2,65.83
6,10250,2,65.83
7,10250,2,65.83
8,10251,1,41.34
9,10251,1,41.34


In [169]:
f = 'Freight ~ C(ShipVia)'
lm = ols(f, q5).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                  sum_sq      df         F    PR(>F)
C(ShipVia)  2.360358e+05     2.0  6.706037  0.001249
Residual    3.787253e+07  2152.0       NaN       NaN


In [179]:
x = """SELECT od.Quantity, od.Discount, o.OrderDate, o.ShippedDate, p.CategoryId, ca.CategoryName
       FROM OrderDetail od
       JOIN [Order] AS o ON od.OrderId = o.Id
       JOIN Product AS p ON od.ProductId = p.Id
       JOIN Category AS ca ON p.CategoryId = ca.Id
       WHERE o.ShippedDate IS NOT NULL
       ;"""
q6 = pd.read_sql_query(x, conn)
q6

,Quantity,Discount,OrderDate,ShippedDate,CategoryId,CategoryName
0,12,0.00,2012-07-04,2012-07-16,4,Dairy Products
1,10,0.00,2012-07-04,2012-07-16,5,Grains/Cereals
2,5,0.00,2012-07-04,2012-07-16,4,Dairy Products
3,9,0.00,2012-07-05,2012-07-10,7,Produce
4,40,0.00,2012-07-05,2012-07-10,7,Produce
5,10,0.00,2012-07-08,2012-07-12,8,Seafood
6,35,0.15,2012-07-08,2012-07-12,7,Produce
7,15,0.15,2012-07-08,2012-07-12,2,Condiments
8,6,0.05,2012-07-08,2012-07-15,5,Grains/Cereals
9,15,0.05,2012-07-08,2012-07-15,5,Grains/Cereals


In [180]:
from datetime import date
q6.OrderDate = pd.to_datetime(q6.OrderDate)
q6.ShippedDate = pd.to_datetime(q6.ShippedDate)
for idx, x in enumerate(q6.OrderDate):
    q6.loc[idx, 'Daysbeforeship'] = (q6.iloc[idx]['ShippedDate'] - q6.iloc[idx]['OrderDate']).days
q6.head()

,Quantity,Discount,OrderDate,ShippedDate,CategoryId,CategoryName,Daysbeforeship
0,12,0.0,2012-07-04,2012-07-16,4,Dairy Products,12.0
1,10,0.0,2012-07-04,2012-07-16,5,Grains/Cereals,12.0
2,5,0.0,2012-07-04,2012-07-16,4,Dairy Products,12.0
3,9,0.0,2012-07-05,2012-07-10,7,Produce,5.0
4,40,0.0,2012-07-05,2012-07-10,7,Produce,5.0


In [191]:
z = q6['Quantity'].value_counts()
z1 = z.to_dict()
q6['Count'] = q6['Quantity'].map(z1)
for idx, x in enumerate(q6.Count):
    if x < 4:
        if q6.Quantity[idx] == 1:
            pass
        else:
            q6.drop(index = idx, axis = 0, inplace=True)
q6 = q6.reset_index(drop=True)
q6.head()

,Quantity,Discount,OrderDate,ShippedDate,CategoryId,CategoryName,Daysbeforeship,Count
0,12,0.0,2012-07-04,2012-07-16,4,Dairy Products,12.0,90
1,10,0.0,2012-07-04,2012-07-16,5,Grains/Cereals,12.0,174
2,5,0.0,2012-07-04,2012-07-16,4,Dairy Products,12.0,67
3,9,0.0,2012-07-05,2012-07-10,7,Produce,5.0,30
4,40,0.0,2012-07-05,2012-07-10,7,Produce,5.0,111


In [195]:
f = 'Daysbeforeship ~ C(CategoryId)'
lm = ols(f, q6).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                     sum_sq      df         F    PR(>F)
C(CategoryId)    231.434185     7.0  0.737923  0.639805
Residual       91534.919789  2043.0       NaN       NaN


In [202]:
for ind, x in enumerate(q6.Discount):
    if x == 0:
        q6.loc[ind, 'is_Discount'] = False
    else:
        q6.loc[ind, 'is_Discount'] = True
display(q6.sample(5))
f = 'Daysbeforeship ~ C(is_Discount)'
lm = ols(f, q6).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

,Quantity,Discount,OrderDate,ShippedDate,CategoryId,CategoryName,Daysbeforeship,Count,is_Discount
155,10,0.00,2012-09-16,2012-09-23,8,Seafood,7.0,174,False
822,30,0.00,2013-06-04,2013-06-10,5,Grains/Cereals,6.0,191,False
1641,20,0.15,2014-02-11,2014-02-20,3,Confections,9.0,243,True
1354,15,0.00,2013-12-08,2013-12-15,4,Dairy Products,7.0,166,False
569,25,0.00,2013-03-05,2013-03-14,1,Beverages,9.0,80,False


                      sum_sq      df         F    PR(>F)
C(is_Discount)     34.637258     1.0  0.773688  0.379182
Residual        91731.716715  2049.0       NaN       NaN


In [205]:
f = 'Daysbeforeship ~ Quantity'
lm = ols(f, q6).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                sum_sq      df         F   PR(>F)
Quantity      1.444071     1.0  0.032244  0.85751
Residual  91764.909902  2049.0       NaN      NaN
